# mAP Caluculate

## Yolov8 Predict

In [ ]:
import os
from ultralytics import YOLO

model = YOLO('C:/Users/AI-Testing-Laptop1/runs/detect_bef/train5/weights/best.pt')

os.environ['KMP_DUPLICATE_LIB_OK']='True'

#test_img1 ~ 9 Memory issue로 9번 나눠서 돌려야함
result = model.predict(source='C:/Users/AI-Testing-Laptop1/Yolov8/test/test_img9/',save=True, save_txt=True, save_conf=True)

## Result convert to List

In [3]:
import os
 
pred_dir = 'C:/Users/AI-Testing-Laptop1/runs/detect/predict/labels'
true_dir = 'C:/Users/AI-Testing-Laptop1/Yolov8/test/labels'
 
file_list = os.listdir(pred_dir)

print(len(file_list))

preds = []

for name in file_list:
    f = open(pred_dir+'/'+name, 'r')
    lines = f.readlines()
    for line in lines:
        temp = line.split()
        temp[0] = int(temp[0])
        for i in range(5):
            temp[i+1] = float(temp[i+1])
    preds.append(temp)
    f.close()
    
print(len(preds))

trues = []
for name in file_list:
    f = open(true_dir+'/'+name, 'r')
    lines = f.readlines()
    for line in lines:
        temp = line.split()
        temp[0] = int(temp[0])
        for i in range(4):
            temp[i+1] = float(temp[i+1])
    trues.append(temp)
    f.close()
    
print(len(trues))

9884
9884
9884


## IoU Calcuate and conf-tp extract

In [4]:
import pandas as pd

tp_list = []
fp_list = []
confidence = []

account = 0

threshold = 0.95

for k in range(5):
 
    tmp_conf = []
    tmp_tp = []
    tmp_fp = []
    
    for i in range(len(preds)):
        
        if trues[i][0] == k:
            pred_box = []
            pred_box.append(preds[i][1]-preds[i][3]/2) #p_x_min
            pred_box.append(preds[i][1]+preds[i][3]/2) #p_x_max
            pred_box.append(preds[i][2]-preds[i][4]/2) #p_y_min
            pred_box.append(preds[i][2]+preds[i][4]/2) #p_y_max

            true_box = []
            true_box.append(trues[i][1]-trues[i][3]/2) #t_x_min
            true_box.append(trues[i][1]+trues[i][3]/2) #t_x_max
            true_box.append(trues[i][2]-trues[i][4]/2) #t_y_min
            true_box.append(trues[i][2]+trues[i][4]/2) #t_y_max

            intersection_x_length = (min(pred_box[1], true_box[1]) - max(pred_box[0], true_box[0]))
            intersection_y_length = (min(pred_box[3], true_box[3]) - max(pred_box[2], true_box[2]))

            pred_box_x_length = (pred_box[1] - pred_box[0])
            pred_box_y_length = (pred_box[3] - pred_box[2])

            true_box_x_length = (true_box[1] - true_box[0])
            true_box_y_length = (true_box[3] - true_box[2])

            intersect_area = intersection_x_length * intersection_y_length
            union_area = pred_box_x_length * pred_box_y_length + true_box_x_length * true_box_y_length - intersect_area
            iou_value = intersect_area/union_area

            account += 1 if (preds[i][0]==trues[i][0]) else 0
            tmp_var = 1 if (iou_value>threshold and preds[i][0]==trues[i][0]) else 0
            tmp_tp.append(tmp_var)
            tmp_fp.append(1 - tmp_var)

            tmp_conf.append(preds[i][5])
            
    tp_list.append(tmp_tp)
    fp_list.append(tmp_fp)
    confidence.append(tmp_conf)

print(account)
    
dfs = []

for k in range(5):
    df = pd.DataFrame({'conf':confidence[k],'tp':tp_list[k],'fp':fp_list[k]}) 
    dfs.append(df)
    dfs[k] = dfs[k].sort_values('conf',ascending = False)
    
for k in range(5):
    
    tp_acc = []
    fp_acc = []
    tp_sum = 0
    fp_sum = 0
    precision = []
    recall = []
    
    for i in range(len(confidence[k])):
        tp_sum = tp_sum + dfs[k].loc[i][1]
        tp_acc.append(tp_sum)
        fp_sum = fp_sum + dfs[k].loc[i][2]
        fp_acc.append(fp_sum)
        precision.append(tp_sum/(i+1))
        recall.append(tp_sum/len(confidence[k]))

    dfs[k]['acc tp'] = tp_acc
    dfs[k]['acc fp'] = fp_acc
    dfs[k]['precision'] = precision
    dfs[k]['recall'] = recall


9236


## Sklearn AP cal, mAP cal

In [ ]:
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import average_precision_score, precision_recall_curve

names = ['Rock bream','Olive flounder','Korea rockfish','Red seabream','Black porgy']

aps = []

for k in range(5):

    truths = np.array(dfs[k]['tp'])
    scores = np.array(dfs[k]['conf'])

    ap = average_precision_score(truths, scores)
    ap_r = int(ap*1000)/1000
    aps.append(ap_r)
    print(names[k] + '클래스의 평균 정밀도(AP)는 '+str(ap_r)+'입니다.')

    precision, recall, _ = precision_recall_curve(truths, scores)
    plt.plot(recall, precision,lw=2)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-recall curve')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.legend(loc='lower left')
    plt.savefig('prcurve.png')
    plt.show()
    
print('IoU가 ' + str(threshold) + '보다 클 때 mAP 는 ' + str(int(sum(aps)*200)/1000))